In [1]:
import sys

In [2]:
sys.path.append('../..')

In [3]:
from datetime import datetime

In [4]:
from app import db
from models import Type as DBType
from models import SDG

139745318568320: loading merged_into_institutions_dict
139745318568320: loading valid concept IDs
139745318568320: loading valid topic IDs


In [6]:
d = """
reference-entry	Entries within reference materials, such as encyclopedias and dictionaries.
standard	Standards developed by Standards Development Organizations (SDOs) or consortia.
dataset	Dataset records capture information about one or more database records or collections.
erratum	Corrections of existing works.
paratext	Paratext are works that represent stuff that is *about* the venue (such as a journal)—rather than a scholarly work properly speaking. These include things like front-covers, back-covers, tables of contents, and the journal itself
article	Research works that generally contains original research or reviews existing research. This includes journal articles, conference articles, and preprint articles.
editorial	Generally short works written by the editors of journals or other publications.
grant	Works associated with research grants, which provide funding to support research.
peer-review	Reviews, reports, or comments about another work.
report	Technical reports and working papers.
letter	These are typically letters to the editors at journals, though it may include other types of letters.
book-chapter	A single chapter within a book. It is common to publish original research as book chapters in certain fields, such as computer science.
dissertation	Dissertations and theses are works representing documents submitted by someone in completion of an academic degree or professional qualification.
other	Works from Crossref marked with type "other".
book	Scholarly books including reference books and monographs.
"""
d = [line.split('\t') for line in d.splitlines() if line]

### SDGs

In [17]:
now = datetime.utcnow().isoformat()
for type_id, description in d:
    t = db.session.query(DBType).filter_by(type_id=type_id).one()
    t.description = description
    t.updated_date = now
    db.session.add(t)
db.session.commit()

In [6]:
d = """
1	No poverty	End poverty in all its forms everywhere.
2	Zero hunger	End hunger, achieve food security and improved nutrition and promote sustainable agriculture.
3	Good health and well-being	Ensure healthy lives and promote well-being for all at all ages.
4	Quality education	Ensure inclusive and equitable quality education and promote lifelong learning opportunities for all.
5	Gender equality	Achieve gender equality and empower all women and girls
6	Clean water and sanitation	Ensure availability and sustainable management of water and sanitation for all.
7	Affordable and clean energy	Ensure access to affordable, reliable, sustainable and modern energy for all.
8	Decent work and economic growth	Promote sustained, inclusive and sustainable economic growth, full and productive employment and decent work for all.
9	Industry, innovation and infrastructure	Build resilient infrastructure, promote inclusive and sustainable industrialization and foster innovation.
10	Reduced inequalities	Reduce inequality within and among countries.
11	Sustainable cities and communities	Make cities and human settlements inclusive, safe, resilient and sustainable.
12	Responsible consumption and production	Ensure sustainable consumption and production patterns.
13	Climate action	Take urgent action to combat climate change and its impacts.
14	Life below water	Conserve and sustainably use the oceans, seas and marine resources for sustainable development.
15	Life on land	Protect, restore and promote sustainable use of terrestrial ecosystems, sustainably manage forests, combat desertification, and halt and reverse land degradation and halt biodiversity loss.
16	Peace, justice, and strong institutions	Promote peaceful and inclusive societies for sustainable development, provide access to justice for all and build effective, accountable and inclusive institutions at all levels.
17	Partnerships for the goals	Strengthen the means of implementation and revitalize the Global Partnership for Sustainable Development.
"""
d = [line.split('\t') for line in d.splitlines() if line]

In [9]:
len(d)

17

In [10]:
now = datetime.utcnow().isoformat()
for sdg_id, sdg_name, description in d:
    t = db.session.query(SDG).filter_by(sdg_id=sdg_id).one()
    t.description = description
    t.updated_date = now
    db.session.add(t)
db.session.commit()

In [6]:
from scripts.wikidata_ror.response import fetch_wikidata_response
import requests

In [6]:
def recurse_find_wikidata_url(x):
    try:
        if x.startswith('http') and 'wikidata' in x:
            return x
        else:
            return None
    except AttributeError:
        try:
            for k in x.keys():
                r = recurse_find_wikidata_url(x[k])
                if r:
                    return r
        except AttributeError:
            for item in x:
                r = recurse_find_wikidata_url(item)
                if r:
                    return r

def get_wikidata_url_from_sdg(sdg):
    url = sdg.un_metadata_id
    headers = {"Accept": "application/ld+json"}
    r = requests.get(url, headers=headers)
    un_metadata = r.json()[0]
    return recurse_find_wikidata_url(un_metadata)

In [7]:
sdgs = db.session.query(SDG).all()
sdg_wikidata_urls = {
    sdg.sdg_id: get_wikidata_url_from_sdg(sdg)
    for sdg in sdgs
}

In [8]:
sdg_wikidata_urls

{17: 'http://www.wikidata.org/entity/Q53581254',
 3: 'http://www.wikidata.org/entity/Q50216838',
 10: 'http://www.wikidata.org/entity/Q53581222',
 13: 'http://www.wikidata.org/entity/Q53581236',
 4: 'http://www.wikidata.org/entity/Q53581209',
 7: 'http://www.wikidata.org/entity/Q53581213',
 14: 'http://www.wikidata.org/entity/Q53581239',
 11: 'http://www.wikidata.org/entity/Q53581228',
 16: 'http://www.wikidata.org/entity/Q53581248',
 15: 'http://www.wikidata.org/entity/Q53581245',
 1: 'http://www.wikidata.org/entity/Q50214636',
 8: 'http://www.wikidata.org/entity/Q53581217',
 6: 'http://www.wikidata.org/entity/Q48741129',
 5: 'http://www.wikidata.org/entity/Q53581210',
 12: 'http://www.wikidata.org/entity/Q53581232',
 9: 'http://www.wikidata.org/entity/Q53581219',
 2: 'http://www.wikidata.org/entity/Q50215926'}

In [9]:
now = datetime.utcnow().isoformat()
for sdg_id, wikidata_url in sdg_wikidata_urls.items():
    t = db.session.query(SDG).filter_by(sdg_id=sdg_id).one()
    t.wikidata_id = wikidata_url.split('/')[-1]
    t.updated_date = now
    db.session.add(t)
db.session.commit()

In [7]:
from scripts.wikidata_ror.image_url import get_image_url, get_image_thumbnail_url

In [16]:
image_url = get_image_url(t.wikidata_id)
image_thumbnail_url = get_image_thumbnail_url(image_url)

In [17]:
image_thumbnail_url

'https://commons.wikimedia.org/w/index.php?title=Special:Redirect/file/Sustainable Development Goal 2.png&width=300'

In [8]:
try:
    sdgs = db.session.query(SDG).all()
    now = datetime.utcnow().isoformat()
    for sdg in sdgs:
        image_url = get_image_url(sdg.wikidata_id)
        image_thumbnail_url = get_image_thumbnail_url(image_url)
        sdg.image_url = image_url
        sdg.image_thumbnail_url = image_thumbnail_url
        sdg.updated_date = now
        db.session.add(sdg)
    db.session.commit()
finally:
    db.session.close()
